## Step 1: Install Necessary Packages

In [1]:
!#pip install  chart_studio

Now let's load the necessary packages for the whole notebook. You might need to 

In [2]:
# common packages 
import numpy as np 
import os
import copy
from math import *
import matplotlib.pyplot as plt
from functools import reduce
from glob import glob
import pydicom
from skimage import measure, morphology
from skimage.morphology import ball, binary_closing
from skimage.measure import label, regionprops
from scipy.linalg import norm
import scipy.ndimage
from ipywidgets.widgets import * 
import ipywidgets as widgets
import plotly
from plotly.graph_objs import *
import chart_studio
chart_studio.tools.set_credentials_file(username='redwankarimsony', api_key='aEbXWsleQv7PJrAtOkBk')

## Step 2: Loading DICOM Data
Let's set the paths of the dicom files and then we will be able to have a look at them. 

In [3]:
patient_id = 'data'
patient_folder = f'/workspaces/codespaces-jupyter/data/'
data_paths = glob(patient_folder + '/*/*.DCM')
print (f'Total of {len(data_paths)} DICOM images.\nFirst 5 filenames:' )
data_paths[:5]

Total of 0 DICOM images.
First 5 filenames:


[]

In [4]:
def load_scan(paths):
    slices = [pydicom.read_file(path) for path in paths]
    slices.sort(key = lambda x: int(x.InstanceNumber), reverse = True)
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

def get_pixels_hu(scans):
    image = np.stack([s.pixel_array for s in scans])
    image = image.astype(np.int16)
    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    intercept = scans[0].RescaleIntercept
    slope = scans[0].RescaleSlope
    
    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)
        
    image += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

Run the following code to extract DICOM pixels for each slice location and display a single slice:

In [5]:
patient_dicom = load_scan(data_paths)
patient_pixels = get_pixels_hu(patient_dicom)
plt.imshow(patient_pixels[80], cmap=plt.cm.bone)

IndexError: list index out of range

## Step 3: Image Processing
Lets use some thresholding and morphological operations to segment just the lung from the chest:

In [ ]:
def largest_label_volume(im, bg=-1):
    vals, counts = np.unique(im, return_counts=True)
    counts = counts[vals != bg]
    vals = vals[vals != bg]
    if len(counts) > 0:
        return vals[np.argmax(counts)]
    else:
        return None
    
def segment_lung_mask(image, fill_lung_structures=True):
    binary_image = np.array(image >= -700, dtype=np.int8)+1
    labels = measure.label(binary_image)
 
    background_label = labels[0,0,0]
 
    binary_image[background_label == labels] = 2
 
    if fill_lung_structures:
        for i, axial_slice in enumerate(binary_image):
            axial_slice = axial_slice - 1
            labeling = measure.label(axial_slice)
            l_max = largest_label_volume(labeling, bg=0)
 
            if l_max is not None: #This slice contains some lung
                binary_image[i][labeling != l_max] = 1
    binary_image -= 1 #Make the image actual binary
    binary_image = 1-binary_image # Invert it, lungs are now 1
 
    labels = measure.label(binary_image, background=0)
    l_max = largest_label_volume(labels, bg=0)
    if l_max is not None: # There are air pockets
        binary_image[labels != l_max] = 0
 
    return binary_image

In [ ]:
# get masks 
segmented_lungs = segment_lung_mask(patient_pixels, fill_lung_structures=False)
segmented_lungs_fill = segment_lung_mask(patient_pixels, fill_lung_structures=True)
internal_structures = segmented_lungs_fill - segmented_lungs

copied_pixels = copy.deepcopy(patient_pixels)
for i, mask in enumerate(segmented_lungs_fill): 
    get_high_vals = mask == 0
    copied_pixels[i][get_high_vals] = 0
seg_lung_pixels = copied_pixels
# sanity check
f, ax = plt.subplots(1,2, figsize=(10,6))
ax[0].imshow(patient_pixels[80], cmap=plt.cm.bone)
ax[0].axis(False)
ax[0].set_title('Original')
ax[1].imshow(seg_lung_pixels[80], cmap=plt.cm.bone)
ax[1].axis(False)
ax[1].set_title('Segmented')
plt.show()

If it looks like this, then perfect! You just isolated the lung from the rest of the scan. Don’t worry about the masks yet, I will show you some visualizations later on.

## 3-D dimensional Image Plotting in Intractive Displaying


In [ ]:
import imageio
from IPython import display
print('Image slices after consideration of the multiple images')
imageio.mimsave(f'./{patient_id}.gif', patient_pixels, duration=0.1)
display.Image(f'./{patient_id}.gif', format='png')

In [ ]:
print('Lung Segmentation')
imageio.mimsave(f'./{patient_id}.gif', segmented_lungs_fill, duration=0.1)
display.Image(f'./{patient_id}.gif', format='png')

In [ ]:
print('Lung Tissue')
imageio.mimsave(f'./{patient_id}.gif', seg_lung_pixels, duration=0.1)
display.Image(f'./{patient_id}.gif', format='png')